In [ ]:
!pip install -q tensorflow-recommenders

In [ ]:
!pip install numpy==1.18.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.18.4.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for numpy
Failed to build numpy
ERROR: Could not build wheels for numpy, which is required to install pyproject.toml-based projects


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pprint
import pandas as pd
import tempfile
from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [ ]:
class Config: 
    
    embedding_dimension = 32
    
    model_path = "model.tf"
    
config = Config()

In [ ]:
# user data.
df = pd.read_csv('/content/data.csv')
# Features of job_interest.
jobs = pd.read_csv('/content/data2.csv')

In [ ]:
df["skills"] = df['skill_1'].astype(str) +", "+ df['skill_2'].astype(str) +", "+ df['skill_3'].astype(str) +", "+ df['skill_4'].astype(str)

In [ ]:
skills = df[['id', 'name', 'skills', 'job_interest']]
skills

,id,name,skills,job_interest
0,AAJ603,Chris Westphalen,"PHP, Visual Design, Google Cloud, Flutter Deve...",.NET Developer
1,HNA156,Koral Nilges,"Node.js, Information Architecture, Salesforce ...",Advance Data Analyst
2,FND122,Biron Mavin,"C#, Wireframing, Google Cloud, Flutter Develop...",AI Researcher
3,FMD294,Rurik Iwanowicz,"SQL, Usability Testing, DigitalOcean, Android ...",android architect
4,XVU891,Bobbee Foord,"Node.js, Wireframing, AWS, Flutter Development",Android Developer
...,...,...,...,...
495,JCH211,Hendrika Amoss,"React, Decision Trees, Xamarin, Digital Ocean",Web Designer
496,KWK881,Sile Watsham,"Django, Classification, Java, Google Cloud",Web Developer
497,BUX671,Dolf Gavin,"HTML/CSS, Random Forest, Ionic, Alibaba Cloud",Web Service Integration Specialist
498,EYD719,Linnea Houndsom,"Angular, Naive Bayes, Ionic, VMware Cloud",Windows Application Developer


In [ ]:
skills = tf.data.Dataset.from_tensor_slices(dict(skills))
jobs = tf.data.Dataset.from_tensor_slices(dict(jobs))

In [ ]:
# get the first rows of the job_interest dataset
for m in jobs.take(100):
  print(m)

{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc1'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'.NET Developer'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc2'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'Advance Data Analyst'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc3'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'AI Researcher'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc4'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'android architect'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc5'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'Android Developer'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc6'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'Android Engineer'>}
{'id': <tf.Tensor: shape=(), dtype=string, numpy=b'xc7'>, 'job_interest': <tf.Tensor: shape=(), dtype=string, numpy=b'Android Technical developer'>}
{'id': <tf.Tensor: sha

In [ ]:
skills = skills.map(lambda x: {
    "id": x["id"],
    "name": x["name"],
    "job_interest": x["job_interest"]
})
jobs = jobs.map(lambda x: {
    "id": x["id"],
    "job_interest": x["job_interest"]}) 

In [ ]:
skills_array = []
jobs_array = []
for data in skills:
    skills_array.append(data['id'].numpy().decode())
for data in jobs:
    jobs_array.append(data['job_interest'].numpy().decode())

#skills_array = tf.squeeze(skills).numpy().astype(str).tolist()
#jobs_array = tf.squeeze(jobs).numpy().astype(str).tolist()

user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(skills_array)

jobs_titles_vocabulary = tf.keras.layers.StringLookup()
jobs_titles_vocabulary.adapt(jobs_array)

In [ ]:
class JobLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.
 
  def __init__(
      self,
      user_model: tf.keras.Model,
      jobs_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()
 
    # Set up user and job_interest representations.
    self.user_model = user_model
    self.jobs_model = jobs_model
 
    # Set up a retrieval task.
    self.task = task
 
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
 
    user_embeddings = self.user_model(features["id"])
    jobs_embeddings = self.jobs_model(features["job_interest"])
 
    return self.task(user_embeddings, jobs_embeddings)

In [ ]:
# Define user and job_interest models.
user_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='id'),
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
jobs_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='job_interest'),
    jobs_titles_vocabulary,
    tf.keras.layers.Embedding(jobs_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        jobs.batch(128).map(jobs_model)
    )
)

/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['id'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [ ]:
# Create a retrieval model.
model = JobLensModel(user_model, jobs_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))
 
# Train for 3 epochs.
model.fit(skills.batch(4096), epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.5280 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3107.1028 - regularization_loss: 0.0000e+00 - total_loss: 3107.1028
Epoch 2/3
1/1 [==============================] - 0s 155ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0460 - factorized_top_k/top_10_categorical_accuracy: 0.3720 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 3024.2825 - regularization_loss: 0.0000e+00 - total_loss: 3024.2825
Epoch 3/3
1/1 [==============================] - 0s 145ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_acc

In [ ]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.jobs_model)
index.index_from_dataset(
    skills.batch(100).map(lambda x: (x['id'], model.user_model(x['id'])))
)

# Get some recommendations.
ids = tf.constant(["xc45"])
_, titles = index(tf.constant(ids))
for i in range(len(ids)):
    print(f"Recommendations for user id {ids[i].numpy()}: {titles[i].numpy()}")

Recommendations for user id b'xc45': [b'EIG602' b'ZYH407' b'EUM063' b'JUO905' b'ZFT379' b'NSZ668' b'TBG638'
 b'QGJ316' b'JFZ010' b'CZD446']


In [ ]:
def recommend(id):
    # Use brute-force search to set up retrieval using the trained representations.
    index = tfrs.layers.factorized_top_k.BruteForce(model.jobs_model)
    index.index_from_dataset(
        skills.batch(100).map(lambda x: (x['id'], model.user_model(x['id'])))
    )
    
    # Get recommendations for the given user ID.
    ids = tf.constant([id])
    _, titles = index(tf.constant(ids))
    
    # Print the recommendations.
    recommendations = titles.numpy().tolist()
    return recommendations
  

In [ ]:
recommend('xc45')

[[b'EIG602',
  b'ZYH407',
  b'EUM063',
  b'JUO905',
  b'ZFT379',
  b'NSZ668',
  b'TBG638',
  b'QGJ316',
  b'JFZ010',
  b'CZD446']]